In [ ]:
import numpy as np 
import pandas as pd 
import scipy.sparse as sp
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse.linalg import spsolve
from subprocess import check_output
from sklearn.model_selection import train_test_split
from numpy import linalg as LA



In [ ]:
train = pd.read_csv('/Users/lmac/Documents/inf_search/movie-recomendation-fall-2020/train.txt', header = None, sep = "\t")

In [ ]:
train.columns = ["UserId", "FilmId", "Mark"]

In [ ]:
train

In [ ]:
n_users = train['UserId'].nunique()

In [ ]:
n_items = train['FilmId'].max()

In [ ]:
train['Mark'].nunique()

In [ ]:
train['Mark'].unique()

In [ ]:
R = np.zeros((n_users, n_items))


In [ ]:
for row in train.itertuples():
    R[(row[1]-1), (row[2]-1)] = row[3]

In [ ]:
R

In [ ]:
R.shape

In [ ]:
r_lambda = 0.1
nf = 50
alpha = 50

In [ ]:
nu = R.shape[0]
ni = R.shape[1]

X = np.random.rand(nu, nf) * 0.01
Y = np.random.rand(ni, nf) * 0.01

print(X)

In [ ]:
P = np.copy(R)
#P[P > 0] = 1
print(P)

In [ ]:
C = 1 + alpha * R
print(C)

In [ ]:
def loss_function(C, P, xTy, X, Y, r_lambda):
    predict_error = np.square(P - xTy)
    confidence_error = np.sum(C * predict_error)
    regularization = r_lambda * (np.sum(np.square(X)) + np.sum(np.square(Y)))
    total_loss = confidence_error + regularization
    return np.sum(predict_error), confidence_error, regularization, total_loss


In [ ]:
def optimize_user(X, Y, C, P, nu, nf, r_lambda):
    yT = np.transpose(Y)
    for u in range(nu):
        Cu = np.diag(C[u])
        yT_Cu_y = np.matmul(np.matmul(yT, Cu), Y)
        lI = np.dot(r_lambda, np.identity(nf))
        yT_Cu_pu = np.matmul(np.matmul(yT, Cu), P[u])
        X[u] = np.linalg.solve(yT_Cu_y + lI, yT_Cu_pu)

def optimize_item(X, Y, C, P, ni, nf, r_lambda):
    xT = np.transpose(X)
    for i in range(ni):
        Ci = np.diag(C[:, i])
        xT_Ci_x = np.matmul(np.matmul(xT, Ci), X)
        lI = np.dot(r_lambda, np.identity(nf))
        xT_Ci_pi = np.matmul(np.matmul(xT, Ci), P[:, i])
        Y[i] = np.linalg.solve(xT_Ci_x + lI, xT_Ci_pi)

In [ ]:
predict_errors = []
confidence_errors = []
regularization_list = []
total_losses = []

for i in range(15):
    if i!=0:   
        optimize_user(X, Y, C, P, nu, nf, r_lambda)
        optimize_item(X, Y, C, P, ni, nf, r_lambda)
    predict = np.matmul(X, np.transpose(Y))
    predict_error, confidence_error, regularization, total_loss = loss_function(C, P, predict, X, Y, r_lambda)
    
    predict_errors.append(predict_error)
    confidence_errors.append(confidence_error)
    regularization_list.append(regularization)
    total_losses.append(total_loss)
    
    print('----------------step %d----------------' % i)
    print("predict error: %f" % predict_error)
    print("confidence error: %f" % confidence_error)
    print("regularization: %f" % regularization)
    print("total loss: %f" % total_loss)
    
predict = np.matmul(X, np.transpose(Y))
print('final predict')
print([predict])

In [ ]:
predict.shape

In [ ]:
np.sqrt(np.mean((R-predict)**2))

In [ ]:
for i in range(10):
    if i!=0:   
        optimize_user(X, Y, C, P, nu, nf, r_lambda)
        optimize_item(X, Y, C, P, ni, nf, r_lambda)
    predict = np.matmul(X, np.transpose(Y))
    predict_error, confidence_error, regularization, total_loss = loss_function(C, P, predict, X, Y, r_lambda)
    
    predict_errors.append(predict_error)
    confidence_errors.append(confidence_error)
    regularization_list.append(regularization)
    total_losses.append(total_loss)
    
    print('----------------step %d----------------' % i)
    print("predict error: %f" % predict_error)
    print("confidence error: %f" % confidence_error)
    print("regularization: %f" % regularization)
    print("total loss: %f" % total_loss)
    
predict = np.matmul(X, np.transpose(Y))
print('final predict')
print([predict])

In [ ]:
from numpy import linalg as LA
LA.norm(R-predict)

In [ ]:
len(R[R != 0])

In [ ]:
predict[R != 0]

In [ ]:
LA.norm(R[R != 0] - predict[R != 0])

In [ ]:
np.max(R[R != 0] - predict[R != 0])

In [ ]:
predict[0, 32]

In [ ]:
predict[0, 19]

In [ ]:
np.min(predict)

In [ ]:
predict[predict < 1] = 1

In [ ]:
np.min(predict)

In [ ]:
with open("submission_wals.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            n_row = test["UserId"][test_id] - 1
            n_col = test["FilmId"][test_id] - 1
            print(f"{test_id + 1},{predict[n_row, n_col]}", file=write_file)

In [ ]:
def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):


    # Calculate the confidence for each value in our data
    confidence = sparse_data * alpha_val
    
    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape
    
    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))
    
    #Precompute I and lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)
    
    I = sparse.eye(features)
    lI = lambda_val * I
    
    # Start main loop. For each iteration we first compute X and then Y
    for i in range(iterations):
        print('iteration %d of %d' % (i+1, iterations))
        
        # Precompute Y-transpose-Y and X-transpose-X
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        # Loop through all users
        for u in range(user_size):

            # Get the user row.
            u_row = confidence[u,:].toarray() 

            # Calculate the binary preference p(u)
            #p_u = u_row.copy()
            #p_u[p_u != 0] = 1.0
            p_u = u_row.copy()
            p_u = p_u/alpha_val
            
            # Calculate Cu and Cu - I
            CuI = sparse.diags(u_row, [0])
            Cu = CuI + Y_I

            # Put it all together and compute the final formula
            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)

    
        for i in range(item_size):

            # Get the item column and transpose it.
            i_row = confidence[:,i].T.toarray()

            # Calculate the binary preference p(i)
            #p_i = i_row.copy()
            #p_i[p_i != 0] = 1.0
            p_i = i_row.copy()
            p_i = p_i/alpha_val
            
            # Calculate Ci and Ci - I
            CiI = sparse.diags(i_row, [0])
            Ci = CiI + X_I

            # Put it all together and compute the final formula
            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

    return X, Y

In [ ]:
data_sparse = sparse.csr_matrix(R)

In [ ]:
user_vecs, item_vecs = implicit_als(data_sparse, iterations=10, features=6, alpha_val=40)


In [ ]:
user_vecs

In [ ]:
predict = user_vecs.dot(item_vecs.T).toarray()



In [ ]:
LA.norm(R[R != 0] - predict[R != 0])

In [ ]:
def nonzeros(m, row):
    for index in range(m.indptr[row], m.indptr[row+1]):
        yield m.indices[index], m.data[index]
      
      
def implicit_als_cg(Cui, features=100, iterations=50, lambda_val=0.01):
    user_size, item_size = Cui.shape

    X = np.random.rand(user_size, features) * 0.01
    Y = np.random.rand(item_size, features) * 0.01

    Cui, Ciu = Cui.tocsr(), Cui.T.tocsr()

    for iteration in range(iterations):
        #if (iteration + 1) % 100 == 0:
        print('iteration %d of %d' % (iteration+1, iterations))
        least_squares_cg(Cui, X, Y, lambda_val)
        least_squares_cg(Ciu, Y, X, lambda_val)
    
    return sparse.csr_matrix(X), sparse.csr_matrix(Y)
  

def least_squares_cg(Cui, X, Y, lambda_val, cg_steps=3):
    users, features = X.shape
    
    YtY = Y.T.dot(Y) + lambda_val * np.eye(features)

    for u in range(users):

        x = X[u]
        r = -YtY.dot(x)

        for i, confidence in nonzeros(Cui, u):
            r += (confidence - (confidence - 1) * Y[i].dot(x)) * Y[i]

        p = r.copy()
        rsold = r.dot(r)

        for it in range(cg_steps):
            Ap = YtY.dot(p)
            for i, confidence in nonzeros(Cui, u):
                Ap += (confidence - 1) * Y[i].dot(p) * Y[i]

            alpha = rsold / p.dot(Ap)
            x += alpha * p
            r -= alpha * Ap

            rsnew = r.dot(r)
            p = r + (rsnew / rsold) * p
            rsold = rsnew

        X[u] = x



In [ ]:
alpha_val_list = [100]
features_list = [250, 300]
lambda_val_list = [0.01, 0.1]
a = 1

for alpha_val in alpha_val_list:
    for features1 in features_list:
            for lambda_val1 in lambda_val_list:
                conf_data = (data_sparse * alpha_val).astype('double')
                user_vecs, item_vecs = implicit_als_cg(conf_data, features1, 200, lambda_val1)
                item_vecs = item_vecs.toarray()
                # Calculate the vector norms
                item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))

                final_score = []

                with open("submission_als" + str(a) + ".csv", 'w') as write_file:
                    print("Id,Score", file=write_file)
                    for test_id in range(len(test)):
                        user_id = test["UserId"][test_id]
                        find_film = test["FilmId"][test_id]            
                        scores = item_vecs.dot(item_vecs[find_film - 1].T) / item_norms
                        scores = scores * mask[user_id - 1, :]
                        num_view = np.sum(scores > 0)
                        if num_view == 0:
                            cur_score = 5
                        else:
                            max_sim = np.argsort(-scores)[:min(num_view, 10)]
                            cur_score = np.mean(R[user_id - 1, max_sim])
                    final_score.append(cur_score)        
                    print(f"{test_id + 1},{cur_score}", file=write_file)
                    
                predictions = np.array(final_score)
                rmse = np.sqrt(np.mean((predictions-targets)**2))
                a = a + 1    
                print(alpha_val, features1, lambda_val1, rmse)



In [ ]:
import matplotlib.pyplot as plt
def runALS(A, R):
    '''
    Runs Alternating Least Squares algorithm in order to calculate matrix.
    :param A: User-Item Matrix with ratings
    :param R: User-Item Matrix with 1 if there is a rating or 0 if not
    :param n_factors: How many factors each of user and item matrix will consider
    :param n_iterations: How many times to run algorithm
    :param lambda_: Regularization parameter
    :return:
    '''
    print("Initiating ")
    lambda_ = 0.1; n_factors = 40; n, m = A.shape; n_iterations = 20
    Users = 5 * np.random.rand(n, n_factors)
    Items = 5 * np.random.rand(n_factors, m)

    def get_error(A, Users, Items, R):
        # This calculates the MSE of nonzero elements
        return np.sum((R * (A - np.dot(Users, Items))) ** 2) / np.sum(R)

    MSE_List = []

    print("Starting Iterations")
    for iter in range(n_iterations):
        for i, Ri in enumerate(R):
            Users[i] = np.linalg.solve(np.dot(Items, np.dot(np.diag(Ri), Items.T)) + lambda_ * np.eye(n_factors),
                                       np.dot(Items, np.dot(np.diag(Ri), A[i].T))).T
        print("Error after solving for User Matrix:", get_error(A, Users, Items, R))

        for j, Rj in enumerate(R.T):
            Items[:,j] = np.linalg.solve(np.dot(Users.T, np.dot(np.diag(Rj), Users)) + lambda_ * np.eye(n_factors),
                                     np.dot(Users.T, np.dot(np.diag(Rj), A[:, j])))
        print("Error after solving for Item Matrix:", get_error(A, Users, Items, R))

        MSE_List.append(get_error(A, Users, Items, R))
        print('%sth iteration is complete...' % iter)

    print(MSE_List)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(range(1, len(MSE_List) + 1), MSE_List); plt.ylabel('Error'); plt.xlabel('Iteration')
    plt.title('Python Implementation MSE by Iteration \n with %d users and %d movies' % A.shape);
    plt.savefig('Python MSE Graph.pdf', format='pdf')
    plt.show()


In [ ]:
PP = np.copy(R)
PP[PP > 0] = 1

runALS(R, PP)

In [ ]:
alg_res = pd.read_csv('/Users/lmac/Documents/inf_search/submission_als1.csv')

predictions = alg_res['Score'].to_numpy()
rmse = np.sqrt(np.mean((predictions-targets)**2))
rmse            

In [ ]:
alpha_val = 100
conf_data = (data_sparse * alpha_val).astype('double')
user_vecs, item_vecs = implicit_als_cg(conf_data, iterations=100, features=250, lambda_val=0.5)



In [ ]:
predict = user_vecs.dot(item_vecs.T).toarray()



In [ ]:
predict = user_vecs.dot(item_vecs.T)



In [ ]:
predict

In [ ]:
np.min(predict)

In [ ]:
LA.norm(mask[mask != 0] - predict[R != 0])

In [ ]:
def recommend(user_id, data_sparse, user_vecs, item_vecs, num_items=10):

    # This is where we calculate the recommendation by taking the 
    # dot-product of the user vectors with the item vectors.
    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()
    
    return rec_vector


In [ ]:
a = recommend(1, data_sparse, user_vecs, item_vecs, num_items=10)

In [ ]:
predict = user_vecs.dot(item_vecs.T).toarray()

In [ ]:
predict

In [ ]:
np.max(predict)

In [ ]:
np.min(predict)

In [ ]:
test = pd.read_csv('/Users/lmac/Documents/inf_search/movie-recomendation-fall-2020/test.txt', header = None, sep = "\t")

In [ ]:
test.columns = ["UserId", "FilmId"]

In [ ]:
test

In [ ]:
with open("submission_als.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            n_row = test["UserId"][test_id] - 1
            n_col = test["FilmId"][test_id] - 1
            print(f"{test_id + 1},{predict[n_row, n_col]}", file=write_file)

In [ ]:
test_id

In [ ]:
len(test)

In [ ]:
test.iloc[[test_id]]

In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit


# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((train['Mark'].astype(float), (train['FilmId'], train['UserId'])))
sparse_user_item = sparse.csr_matrix((train['Mark'].astype(float), (train['UserId'], train['FilmId'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=150, regularization=0.05, iterations=50, calculate_training_loss=True)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 100
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)


 

In [ ]:


# Create recommendations for user with id 1
user_id = 1

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item, N = n_items)

In [ ]:
df = pd.DataFrame(recommended, columns =['Id', 'Score'])
df[df['Id'] == 20]['Score']

In [ ]:
min(df['Score'])

In [ ]:
max(df['Score'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 5))

In [ ]:
user_id = 1
recommended = model.recommend(user_id, sparse_user_item, N = n_items)
find_film = 20
df = pd.DataFrame(recommended, columns =['Id', 'Score'])
df[['Score']] = scaler.fit_transform(df[['Score']])
df

In [ ]:
recommended

In [ ]:
cur_score = df.loc[df['Id'] == 1, 'Score'].iloc[0]

In [ ]:
cur_score

In [ ]:
similar = model.similar_items(1, n_items)



In [ ]:
df = pd.DataFrame(similar, columns =['Id', 'Score'])

df

In [ ]:
min(df['Score'])

In [ ]:
max(df['Score'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

with open("submission_als5.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            recommended = model.recommend(user_id, sparse_user_item, N = n_items)
            find_film = test["FilmId"][test_id]
            df = pd.DataFrame(recommended, columns =['Id', 'Score'])
            df[['Score']] = scaler.fit_transform(df[['Score']])
            cur_score = df.loc[df['Id'] == find_film, 'Score'].iloc[0]
            print(f"{test_id + 1},{cur_score}", file=write_file)

In [ ]:
mask = np.copy(R)
mask[mask > 0] = 1

mask

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

with open("submission_als11.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            find_film = test["FilmId"][test_id]            
            similar = model.similar_items(find_film, n_items)
            df = pd.DataFrame(similar, columns =['Id', 'Score'])
            df[['Score']] = scaler.fit_transform(df[['Score']])
            cur_scores = df['Score'].values * mask[user_id - 1, :]
            #weights = R[user_id - 1, :]
            num_view = np.sum(cur_scores > 0)
            if num_view == 0:
                 cur_score = 5
            else:
                max_sim = np.argsort(-cur_scores)[:min(num_view, 10)]
                #cur_score = np.dot(cur_score,weights)
                cur_score = np.mean(R[user_id - 1, max_sim])
            print(f"{test_id + 1},{cur_score}", file=write_file)

In [ ]:
item_vecs = item_vecs.toarray()
user_vecs = user_vecs.toarray()




In [ ]:

# Calculate the vector norms
item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))
final_score = []

with open("submission_als1.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            find_film = test["FilmId"][test_id]            
            scores = item_vecs.dot(item_vecs[find_film - 1].T) / item_norms
            scores = scores * mask[user_id - 1, :]
            num_view = np.sum(scores > 0)
            if num_view == 0:
                 cur_score = 5
            else:
                max_sim = np.argsort(-scores)[:min(num_view, 10)]
                cur_score = np.mean(R[user_id - 1, max_sim])
            final_score.append(cur_score)     
            print(f"{test_id + 1},{cur_score}", file=write_file)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity_items = cosine_similarity(item_vecs)

In [ ]:
cosine_similarity_items.shape

In [ ]:
cosine_similarity_items[0, :]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 5))
recommend_score = []
recommend_score_orig = []


with open("submission_als2.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id] - 1
            recommended = user_vecs[user_id,:].dot(item_vecs.T)
            find_film = test["FilmId"][test_id] - 1
            recommend_score_orig.append(recommended[find_film])
            recommended = scaler.fit_transform(recommended.reshape(-1, 1))
            cur_score = recommended[find_film].item(0)
            recommend_score.append(cur_score)
            print(f"{test_id + 1},{cur_score}", file=write_file)
            
recommend_score = np.array(recommend_score)
recommend_score_orig = np.array(recommend_score_orig)
            

In [ ]:
recommend_score

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 5))
predict = user_vecs.dot(item_vecs.T)
predict_orig = predict.copy()
predict = scaler.fit_transform(predict)
recommend_score = []
recommend_score_orig = []


with open("submission_als2.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id] - 1
            find_film = test["FilmId"][test_id] - 1
            recommend_score_orig.append(predict_orig[user_id, find_film])
            cur_score = predict[user_id, find_film]
            recommend_score.append(cur_score)
            print(f"{test_id + 1},{cur_score}", file=write_file)
            
recommend_score = np.array(recommend_score)
recommend_score_orig = np.array(recommend_score_orig)



In [ ]:
recommend_score

In [ ]:
final_score = []
num_view_list = []
final_top = []
cosine_similarity_items = cosine_similarity(item_vecs)

with open("submission_als1.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            find_film = test["FilmId"][test_id]            
            scores = cosine_similarity_items[find_film - 1, :]
            scores = scores * mask[user_id - 1, :]
            num_view = np.sum(scores > 0)
            num_view_list.append(num_view)
            if num_view == 0:
                cur_score = 5
                final_top.append([5])   
            else:
                max_sim = np.argsort(-scores)[:min(num_view, 10)]
                cur_score = np.mean(R[user_id - 1, max_sim])
                final_top.append(R[user_id - 1, max_sim])
            final_score.append(cur_score)
            
            print(f"{test_id + 1},{cur_score}", file=write_file)
final_score = np.array(final_score)
num_view_list = np.array(num_view_list)
predictions = np.array(final_score)

np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
final_score = []
num_view_list = []
with open("submission_als1.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            find_film = test["FilmId"][test_id]            
            scores = cosine_similarity_items[find_film - 1, :]
            scores = scores * mask[user_id - 1, :]
            num_view = np.sum(scores != 0)
            num_view_list.append(num_view)
            if num_view == 0:
                 cur_score = 5
            else:
                max_sim = np.argsort(-scores)[:min(num_view, num_view//10)]
                cur_score = np.mean(R[user_id - 1, max_sim])
            final_score.append(cur_score)     
            print(f"{test_id + 1},{cur_score}", file=write_file)
final_score = np.array(final_score)
num_view_list = np.array(num_view_list)

In [ ]:
user_vecs = user_vecs.toarray()



In [ ]:
# Calculate the vector norms
final_score_users = []
cosine_similarity_users = cosine_similarity(user_vecs)
scaler = MinMaxScaler(feature_range=(0, 1))
final_top_users = []

with open("submission_als1.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id] -1 
            find_film = test["FilmId"][test_id] - 1            
            scores = cosine_similarity_users[user_id, :]
            scores = scores * mask[:, find_film]
            num_view = np.sum(scores > 0)
            if num_view == 0:
                cur_score = 5
                final_top_users.append([5])   
            else:
                max_sim = np.argsort(-scores)[:min(num_view, 30)]
                cur_score = np.mean(R[max_sim, find_film])
                final_top_users.append(R[max_sim, find_film])
            final_score_users.append(cur_score)
            
final_score_users = np.array(final_score_users)

predictions = final_score_users

np.sqrt(np.mean((predictions-targets)**2))            

In [ ]:
# Calculate the vector norms
final_score_users = []
cosine_similarity_users = cosine_similarity(user_vecs)
scaler = MinMaxScaler(feature_range=(0, 1))


with open("submission_als1.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id] -1 
            find_film = test["FilmId"][test_id] - 1            
            scores = cosine_similarity_users[user_id, :]
            scores = scores[mask[:, find_film] > 0]
            num_view = len(scores)
            if num_view == 0:
                 cur_score = 5
            else:
                if num_view > 1:
                    scores =  scaler.fit_transform(scores.reshape(-1, 1))
                cur_score = np.mean(R[mask[:, find_film] > 0, find_film] * scores)
            final_score_users.append(cur_score)     
            print(f"{test_id + 1},{cur_score}", file=write_file)
            
final_score_users = np.array(final_score_users)

predictions = final_score_users

np.sqrt(np.mean((predictions-targets)**2))  

In [ ]:
final_score_users

In [ ]:
scores[mask[:, find_film] > 0]

In [ ]:
cosine_similarity_users[0, :].shape

In [ ]:
R[mask[:, find_film] > 0, find_film]

In [ ]:
final_score

In [ ]:
num_view_list

In [ ]:
sorted(final_top, key=lambda x: x[0])

In [ ]:
df = pd.DataFrame()
df['final_score'] = final_score
df['target'] = targets
df['diff_1'] = final_score - targets
df['num_view'] = num_view_list
df['recommend_score'] = recommend_score
df['diff_2'] = recommend_score - targets
df['recommend_score_orig'] = recommend_score_orig
df['final_score_users'] = final_score_users
df['diff_3'] = final_score_users - targets
#df['cross_score'] = 3/4 * final_score + 1/4 * recommend_score
df['final_top_items'] = final_top
df['final_top_users'] = final_top_users
df['top_items_mark'] = [item[0] for item in final_top]
df['top_users_mark'] = [item[0] for item in final_top_users]
df

In [ ]:
df.sort_values(by='diff_1', ascending=False).head(100)


In [ ]:
df.loc[(df['top_users_mark'] == 1) & (df['target'] != 1)]

In [ ]:
df.loc[(df['top_items_mark'] == 1) & (df['target'] != 1)]

In [ ]:
df.loc[df['top_items_mark'].isin([1, 2]) & df['top_users_mark'].isin([1, 2]) & (df['final_score'] < 2.5)]

In [ ]:
[item[0] for item in final_top_users]

In [ ]:
cross_score = 1/2 * final_score + 1/6 * final_score_users + 1/6 * np.array(df['top_items_mark']) + 1/6 * np.array(df['top_users_mark'])
predictions = cross_score
np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
cross_score = 2/3 * final_score + 1/9 * final_score_users + 1/9 * np.array(df['top_items_mark']) + 1/9 * np.array(df['top_users_mark'])
predictions = cross_score
np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
cross_score = 1/2 * final_score + 1/4 * final_score_users + 1/8 * np.array(df['top_items_mark']) + 1/8 * np.array(df['top_users_mark'])
predictions = cross_score
np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
cross_score = 1/2 * final_score + 1/4 * final_score_users + 1/9 * np.array(df['top_items_mark']) + 1/9 * np.array(df['top_users_mark'])
cross_score = cross_score + 1/36 * np.array(df['recommend_score'])
predictions = cross_score
np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
from sklearn.linear_model import LinearRegression


X = np.c_[final_score, final_score_users, np.array(df['top_items_mark']), np.array(df['top_users_mark']), np.array(df['recommend_score'])]
y = targets

reg = LinearRegression().fit(X, y)


In [ ]:
reg.get_params()

In [ ]:
reg.score(X, y)

In [ ]:
reg.coef_

In [ ]:
reg.intercept_

In [ ]:
cross_score = reg.predict(X)
predictions = cross_score
np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
np.array(df['top_items_mark'])

In [ ]:
np.array(df['top_users_mark'])

In [ ]:
X

In [ ]:
cosine_similarity_users = cosine_similarity(user_vecs)

In [ ]:
final_score = []

with open("submission_als1.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            find_film = test["FilmId"][test_id]            
            scores = cosine_similarity_users[user_id - 1, :]
            scores = scores * mask[:, find_film - 1]
            num_view = np.sum(scores > 0)
            if num_view == 0:
                 cur_score = 5
            else:
                max_sim = np.argsort(-scores)[:min(num_view, 10)]
                cur_score = np.mean(R[user_id - 1, max_sim])
            final_score.append(cur_score)     
            print(f"{test_id + 1},{cur_score}", file=write_file)

In [ ]:
final_score = np.array(final_score)

In [ ]:
targets = np.array(targets) 

In [ ]:
decpart = np.modf(final_score)[0]

In [ ]:
decpart[decpart > 0.9] = 1
decpart[decpart < 0.1] = 0

In [ ]:
predictions = np.modf(final_score)[1] + decpart

np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
np.modf([0, 3.5])

In [ ]:

predictions = np.array(final_score)

targets = right_res

np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
predictions = recommend_score
np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
predictions = cross_score
np.sqrt(np.mean((predictions-targets)**2))

In [ ]:
predictions13

In [ ]:
np.sum(predictions13 < 2)

In [ ]:
np.sum(np.array(targets) < 2)

In [ ]:
sorted(-np.abs(predictions13 - targets))

In [ ]:
user_vecs = user_vecs.toarray()


In [ ]:
user_vecs.shape

In [ ]:
# Calculate the vector norms
user_norms = np.sqrt((user_vecs * user_vecs).sum(axis=1))
final_score = []

with open("submission_als1.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            find_film = test["FilmId"][test_id]            
            scores = user_vecs.dot(user_vecs[user_id - 1].T) / user_norms
            scores = scores * mask[user_id - 1, :]
            num_view = np.sum(scores > 0)
            if num_view == 0:
                 cur_score = 5
            else:
                max_sim = np.argsort(-scores)[:min(num_view, 10)]
                cur_score = np.mean(R[user_id - 1, max_sim])
            final_score.append(cur_score)     
            print(f"{test_id + 1},{cur_score}", file=write_file)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

with open("submission_als1.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id] - 1
            recommended = user_vecs[user_id,:].dot(item_vecs.T)
            find_film = test["FilmId"][test_id] - 1
            recommended = recommended / LA.norm(recommended) 
            cur_score = recommended[find_film]
            print(f"{test_id + 1},{cur_score}", file=write_file)

In [ ]:
user_id = 0
recommended = user_vecs[user_id,:].dot(item_vecs.T)

recommended[19]



In [ ]:
item_vecs

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

with open("submission_als6.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            find_film = test["FilmId"][test_id]            
            similar = model.similar_items(find_film, n_items)
            df = pd.DataFrame(similar, columns =['Id', 'Score'])
            df[['Score']] = scaler.fit_transform(df[['Score']])
            cur_scores = df['Score'].values * mask[user_id - 1, :]
            weights = R[user_id - 1, :]
            cur_score = np.sum(cur_scores * weights) / np.sum(weights)
            print(f"{test_id + 1},{cur_score}", file=write_file)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

with open("submission_als7.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            user_id = test["UserId"][test_id]
            find_film = test["FilmId"][test_id]            
            similar = model.similar_users(user_id, n_users)
            df = pd.DataFrame(similar, columns =['Id', 'Score'])
            df[['Score']] = scaler.fit_transform(df[['Score']])
            cur_scores = df['Score'].values * mask[:, find_film - 1]
            #weights = R[user_id - 1, :]
            max_sim = np.argsort(-cur_scores)[:10]
            #cur_score = np.dot(cur_score,weights)
            cur_score = np.mean(R[user_id - 1, max_sim])
            print(f"{test_id + 1},{cur_score}", file=write_file)

In [ ]:
test

In [ ]:
min(mask.sum(axis=0))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
user_id = 1
find_film = 20
similar = model.similar_items(find_film, n_items)

df = pd.DataFrame(similar, columns =['Id', 'Score'])
df[['Score']] = scaler.fit_transform(df[['Score']])
df

In [ ]:
cur_scores = df['Score'].values * mask[user_id - 1, :]
weights = R[user_id - 1, :]
cur_score = np.sum(cur_scores * weights) / np.sum(weights)
cur_score

In [ ]:
cur_score = np.mean(R[user_id - 1, max_sim])
#cur_score = np.dot(cur_score,weights)
#cur_score = R[user_id - 1, max_sim]
 
cur_score

In [ ]:
cur_scores

In [ ]:
mask[0, :] 

In [ ]:
cur_score = df['Score'].values * mask[0, :] 

In [ ]:
weights = R[0, :]

In [ ]:
np.argmax(cur_score)

In [ ]:
np.dot(cur_score,weights)

In [ ]:
sum(weights > 0)

In [ ]:
alg_res = pd.read_csv('/Users/lmac/Documents/inf_search/submission_als6.csv')

In [ ]:
res1 = pd.read_csv('/Users/lmac/Documents/inf_search/submission_als.csv')

In [ ]:
diff = res1['Score'] - res2['Score']
diff

In [ ]:
np.square(np.sum(diff**2))

In [ ]:
np.sort(diff)

In [ ]:
res2

In [ ]:
df = pd.DataFrame(recommended, columns =['Id', 'Score'])

df

In [ ]:
max(df['Score'])

In [ ]:
min(df['Score'])

In [ ]:
R

In [ ]:
mask = np.round(R > 0)

mask

In [ ]:
R.shape

In [ ]:
n_users, n_items

In [ ]:
from matrix_completion import svt_solve, calc_unobserved_rmse

U = np.random.randn(n_users, 100)
V = np.random.randn(n_items, 100)
R_hat = svt_solve(R, mask)

print("RMSE:", calc_unobserved_rmse(U, V, R_hat, mask))

In [ ]:
U

In [ ]:
V

In [ ]:
V.dot(U.transpose())

In [ ]:
R

In [ ]:
import random
import math

import numpy as np
from scipy.linalg import orth


def _make_lr_matrix(m, n, k):
    L = np.random.randn(m, k)
    R = np.random.randn(k, n)
    return np.dot(L, R)


def _get_masked_matrix(M, omega):
    M_max, M_min = np.max(M), np.min(M)
    M_ = M.copy()
    M_[(1 - omega).astype(np.int16)] = M_max * M_max
    return M_


def _get_V_from_U(M, U, omega):
    column = M.shape[1]
    rank = U.shape[1]
    V = np.empty((rank, column), dtype=M.dtype)

    for j in range(0, column):
        U_ = U.copy()
        U_[(1 - omega[:, j]).astype(np.int16), :] = 0
        V[:, j] = np.linalg.lstsq(U_, M[:, j], rcond=None)[0]
    return V


def _get_err(M, U, V, omega):
    error_matrix = M - np.dot(U, V)
    error_matrix[(1 - omega).astype(np.int16)] = 0
    return np.linalg.norm(error_matrix, 'fro') / np.count_nonzero(omega)


def _split_omega(omega, T):
    omegas = [np.zeros(omega.shape) for t in range(2 * T + 1)]
    row, col = omega.shape
    for i in range(row):
        for j in range(col):
            idx = random.randint(0, 2 * T)
            omegas[idx][i, j] = omega[i, j]
    return omegas


def _init_U(M, omega, p, k, mu):
    M[(1 - omega).astype(np.int16)] = 0
    M = M / p
    U, S, V = np.linalg.svd(M, full_matrices=False)
    print(U.shape)
    U_hat = U.copy()
    print(U_hat.shape)
    clip_threshold = 2 * mu * math.sqrt(k / max(M.shape))
    U_hat[U_hat > clip_threshold] = 0
    print(U_hat.shape)
    U_hat = orth(U_hat)
    print(U_hat.shape)
    print("|U_hat-U|_F/|U|_F:",
          np.linalg.norm(np.subtract(U_hat, U), ord='fro') / np.linalg.norm(
              U, ord='fro'))
    return U_hat


def _solve(M, omega, p, k, T, mu):
    omegas = _split_omega(omega, T)
    U = _init_U(M[:, :], omegas[0], p, k, mu)
    print('')
    V = None
    for t in range(T):
        V = _get_V_from_U(M, U, omegas[t + 1])
        U = _get_V_from_U(M.T, V.T, omegas[T + t + 1].T).T

        err = _get_err(M, U, V, omega)
        print('>> t(%3d):' % t, err)
    print('')
    assert V is not None
    return np.dot(U, V)


def main(M, omega, m, n, k, p, T, mu):
    #M = _make_lr_matrix(m, n, k)
    #omega = np.zeros((m, n))
    #omega[np.random.rand(m, n) <= p] = 1
    cardinality_of_omega = np.count_nonzero(omega)
    omega = omega.astype(np.int16)
    M_rank = np.linalg.matrix_rank(M)
    print("RANK of M        :", M_rank)
    M_ = _get_masked_matrix(M, omega)

    X = _solve(M, omega, p, k, T, mu)
    X_rank = np.linalg.matrix_rank(X)
    print("RANK of X        :", X_rank)

    E = np.subtract(M, X)
    E_train = E.copy()
    np.place(E_train, 1 - omega, 0)
    print('TRAIN RMSE       :',
          np.linalg.norm(E_train, "fro") / cardinality_of_omega)
    E_test = E.copy()
    np.place(E_test, omega, 0)
    print('TEST  RMSE       :',
          np.linalg.norm(E_test, "fro") / (m * n - cardinality_of_omega))

    print("|X-M|_F/|M|_F    :",
          np.linalg.norm(np.subtract(M, X), ord='fro') / np.linalg.norm(
              M, ord='fro'))




In [ ]:
    # Given
    m = n_users
    n = n_items
    p = 0.1

    # Hyper Parameters
    k = 40
    T = 5
    mu = 0.1
    main(R, mask, m, n, k, p, T, mu)

In [ ]:
from d2l import mxnet as d2l
import os
import pandas as pd


In [ ]:
#@save
d2l.DATA_HUB['ml-100k'] = (
    'http://files.grouplens.org/datasets/movielens/ml-100k.zip',
    'cd4dcac4241c8a4ad7badc7ca635da8a69dddb83')

#@save
def read_data_ml100k():
    data_dir = d2l.download_extract('ml-100k')
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names,
                       engine='python')
    num_users = data.user_id.unique().shape[0]
    num_items = data.item_id.unique().shape[0]
    return data, num_users, num_items

In [ ]:
data, num_users, num_items = read_data_ml100k()


In [ ]:
data

In [ ]:
mldat = np.zeros((num_users, num_items))

for row in data.itertuples():
    mldat[(row[1]-1), (row[2]-1)] = row[3]

In [ ]:
mldat

In [ ]:
mldat.item((0, 0))

In [ ]:
right_res = []
for test_id in range(len(test)):
            user_id = test["UserId"][test_id] - 1
            find_film = test["FilmId"][test_id] - 1
            right_res.append(mldat.item((user_id, find_film)))


In [ ]:
right_res

In [ ]:
right_res = np.array(right_res)

In [ ]:
right_res

In [ ]:
alg_res13 = pd.read_csv('/Users/lmac/Documents/inf_search/submission_als1.csv')

predictions13 = alg_res13['Score'].to_numpy()

targets = right_res

np.sqrt(np.mean((predictions13-targets)**2))

In [ ]:
np.argsort(predictions-targets)

In [ ]:
test.values[6749-1]

In [ ]:
sorted(predictions-targets)

In [ ]:
predictions5-targets

In [ ]:
max(predictions11-targets)

In [ ]:
min(predictions11-targets)

In [ ]:
a = predictions[np.argsort(predictions-targets)]

In [ ]:
a

In [ ]:
predictions[predictions != 5]

In [ ]:
right_res[len(right_res) - 1]

In [ ]:
np.sum(R[:, 1254] > 0)

In [ ]:
import os, sys, numpy as np


os.environ["PYSPARK_PYTHON"]="/usr/local/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/local/bin/python3"
os.environ['SPARK_HOME']="/Users/lmac/Downloads/spark-3.0.1-bin-hadoop2.7"
sys.path.append("/Users/lmac/Downloads/spark-3.0.1-bin-hadoop2.7/python/")


from pyspark import SparkConf, SparkContext
conf = (SparkConf().setMaster("local").setAppName("My app").set("spark.executor.memory", "1g"))
from pyspark.sql import SQLContext
from pyspark.sql.dataframe import StructType, StructField, IntegerType, FloatType
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)


In [ ]:
mv_ratings = sc.textFile('/Users/lmac/Documents/inf_search/movie-recomendation-fall-2020/train.txt')

In [ ]:
mv_ratings.first()

In [ ]:
item_user_ratings.first()

In [ ]:
# Creating sparse representation of A matrix with users as rows and items as columns
user_item_ratings = mv_ratings.map(lambda line: (int(line.split('\t')[0]), (int(line.split('\t')[1]), line.split('\t')[2])))
user_item_ratings = user_item_ratings.groupByKey()


In [ ]:
item_user_ratings = mv_ratings.map(lambda line: (int(line.split('\t')[1]), (int(line.split('\t')[0]), line.split('\t')[2])))
item_user_ratings = item_user_ratings.groupByKey()


In [ ]:
lambda_ = sc.broadcast(0.1) # Regularization parameter
n_factors = sc.broadcast(3) # nfactors of User matrix and Item matrix
n_iterations = 20 # How many times to iterate over the user and item matrix calculations.


In [ ]:
Items = item_user_ratings.map(lambda line: (line[0], 5 * np.random.rand(1, n_factors.value)))
print(Items.take(10))

In [ ]:
Items_broadcast = sc.broadcast({
  k: v for (k, v) in Items.collect()
})

In [ ]:
print("Number of Items (columns of A matrix):", item_user_ratings.count())
print("Number of Users (rows of A matrix):", user_item_ratings.count())

In [ ]:
def Update_User(userTuple):

    Itemssquare = np.zeros([n_factors.value,n_factors.value])
    for matrixA_item_Tuple in userTuple[1]:
        itemRow = Items_broadcast.value[matrixA_item_Tuple[0]][0]
        for i in range(n_factors.value):
            for j in range(n_factors.value):
                Itemssquare[i,j] += float(itemRow[i]) * float(itemRow[j])
    leftMatrix = np.linalg.inv(Itemssquare + lambda_.value * np.eye(n_factors.value))
    rightMatrix = np.zeros([1,n_factors.value])
    for matrixA_item_Tuple in userTuple[1]:
        for i in range(n_factors.value):
            rightMatrix[0][i] += Items_broadcast.value[matrixA_item_Tuple[0]][0][i] * float(matrixA_item_Tuple[1])
    newUserRow = np.dot(leftMatrix, rightMatrix.T).T
    return (userTuple[0], newUserRow)

In [ ]:
Users = user_item_ratings.map(Update_User)


In [ ]:
# The item matrix is needed in all partitions when solving for rows of User matrix individually
Users_broadcast = sc.broadcast({
  k: v for (k, v) in Users.collect()
})

In [ ]:
def Update_Item(itemTuple):

    Userssquare = np.zeros([n_factors.value,n_factors.value])
    for matrixA_user_Tuple in itemTuple[1]:
        userRow = Users_broadcast.value[matrixA_user_Tuple[0]][0]
        for i in range(n_factors.value):
            for j in range(n_factors.value):
                Userssquare[i,j] += float(userRow[i]) * float(userRow[j])
    leftMatrix = np.linalg.inv(Userssquare + lambda_.value * np.eye(n_factors.value))
    rightMatrix = np.zeros([1,n_factors.value])
    for matrixA_user_Tuple in itemTuple[1]:
        for i in range(n_factors.value):
            rightMatrix[0][i] += Users_broadcast.value[matrixA_user_Tuple[0]][0][i] * float(matrixA_user_Tuple[1])
    newItemRow = np.dot(leftMatrix, rightMatrix.T).T
    return (itemTuple[0], newItemRow)

In [ ]:
Items = item_user_ratings.map(Update_Item)


In [ ]:
Items_broadcast = sc.broadcast({
  k: v for (k, v) in Items.collect()
})

In [ ]:
def getRowSumSquares(userTuple):
    userRow = Users_broadcast.value[userTuple[0]]
    rowSSE = 0.0
    for matrixA_item_Tuple in userTuple[1]:
        predictedRating = 0.0
        for i in range(n_factors.value):
            predictedRating += userRow[0][i] * Items_broadcast.value[matrixA_item_Tuple[0]][0][i]
        SE = (float(matrixA_item_Tuple[1]) - predictedRating) ** 2
        rowSSE += SE
    return rowSSE

In [ ]:
SSE = user_item_ratings.map(getRowSumSquares).reduce(lambda a, b: a + b)
Count = mv_ratings.count()
MSE = SSE / Count
print("MSE:", MSE)

In [ ]:
for iter in range(n_iterations):
    Users = user_item_ratings.map(Update_User)
    Users_broadcast = sc.broadcast({k: v for (k, v) in Users.collect()})
    Items = item_user_ratings.map(Update_Item)
    Items_broadcast = sc.broadcast({k: v for (k, v) in Items.collect()})
    SSE = user_item_ratings.map(getRowSumSquares).reduce(lambda a, b: a + b)
    MSE = SSE / Count
    print("MSE:", MSE)

In [ ]:
Users.first()

In [ ]:
Items.first()

# Final version

In [ ]:
import numpy as np 

np.random.seed(100)
from matplotlib import pyplot as plt
class ImplicitALS:
    
    def __init__(self, max_epoch=10, embedding_size=15, alpha=10, lamdda=0.1, eps=0.1,
                  mean_decrease=0.85):

        self.max_epoch = max_epoch
        self.embedding_size = embedding_size
        self.alpha = alpha
        self.lamdda = lamdda
        self.eps = eps
        self.mean_decrease = mean_decrease

    def log_confidence(self, R_train):
        return (np.ones(R_train.shape) + self.alpha * np.log(np.ones(R_train.shape) + R_train / self.eps))

    def normalisation(self, R, R_test):
        gr_mask = R > 0
        eq_mask = R == 0
        mean = R[gr_mask].mean()
        R_unbiased = R * gr_mask - gr_mask * mean
        user_bias = (R_unbiased.sum(1) / gr_mask.sum(1)).reshape(-1, 1)
        R_unuserbiased = R_unbiased * gr_mask - gr_mask * user_bias
        gr_mask_sum_0 = gr_mask.sum(0)
        gr_mask_sum_0[gr_mask_sum_0 == 0] = 1
        item_bias = (R_unuserbiased.sum(0) / gr_mask_sum_0).reshape(1, -1)

        gr_mask_test = R_test > 0
        P = R + eq_mask * (~gr_mask_test) * (user_bias + item_bias + mean) * self.mean_decrease + eq_mask * \
            gr_mask_test * (user_bias + item_bias + mean)
        return P, mean, gr_mask, eq_mask

    def calc_loss(self, epoch, result, R_train, R_test, X, Y, C, gamma, beta, mean, gr_mask):

        train_error = np.sqrt(((result * gr_mask - R_train) ** 2).sum() / gr_mask.sum())

        R = R_train.copy()
        if R_test is not None:
            test_error = np.sqrt(((result * (R_test > 0) - R_test) ** 2).sum() / (R_test > 0).sum())
            R += R_test
        else:
            test_error = None

        metric = 1.0/(R.shape[0] * R.shape[1]) * \
                 (C * (R - beta - gamma - result) ** 2).sum() + \
                 self.lamdda * ((X**2).sum() + (Y**2).sum() + (beta**2).sum() + (gamma**2).sum())
        return train_error, test_error, metric

    def fit(self, R_train, R_test=None):

        tr_loss = []
        te_loss = []
        losses = []
        P, mean, gr_mask, eq_mask = self.normalisation(R_train, R_test)

        C = self.log_confidence(R_train)

        laI = np.eye(self.embedding_size + 1, self.embedding_size + 1) * self.lamdda  


        X = np.hstack([np.ones((R_train.shape[0], 1)), np.random.random_sample((R_train.shape[0], self.embedding_size))])
        Y = np.hstack([np.ones((R_train.shape[1], 1)), np.random.random_sample((R_train.shape[1], self.embedding_size))])

        beta = np.zeros((X.shape[0], 1))  
        gamma = np.zeros((X.shape[0], 1))  


        n_user = (R_train + R_test > 0).sum(1)
        n_item = (R_train + R_test > 0).sum(0)


        for epoch in range(self.max_epoch):

            Pgamma = P - gamma
            Cp = C * Pgamma
            for i in range(X.shape[0]): 
                YTCuY_laI = np.dot(C[i,:] * Y.T, Y) + laI * (n_user[i]) 
                YTCuY_laI_inv = np.linalg.inv(YTCuY_laI)
                YTCuY_laI_inv_YT = np.matmul(YTCuY_laI_inv, Y.T)
                X[i, :] = np.matmul(YTCuY_laI_inv_YT, Cp[i, :].reshape(-1, 1)).ravel()

            beta = X[:, 0].copy().reshape(-1, 1)
            X[:, 0] = 1
            Pbeta = P - beta
            Cp = C * Pbeta
            for j in range(Y.shape[0]):  
                
                XTCuX_laI = np.dot(C[:,j] * X.T, X) + laI * (n_item[j]) 
                XTCuX_laI_inv = np.linalg.inv(XTCuX_laI)
                XTCuX_laI_inv_XT = np.matmul(XTCuX_laI_inv, X.T)
                Y[j, :] = np.matmul(XTCuX_laI_inv_XT, Cp[:, j].reshape(-1, 1)).ravel()

            gamma = Y[:, 0].copy().reshape(1, -1)
            Y[:, 0] = 1
            current_result = np.matmul(X[:, 1:], Y[:, 1:].T) + beta + gamma
            current_result_new = current_result.copy()

            current_result_new[current_result_new > 5] = 5
            current_result_new[current_result_new < 1] = 1
            
            
            
            train_error, test_error, loss = self.calc_loss(epoch, current_result_new, R_train, R_test, X, Y, C,
                              gamma, beta, mean,  gr_mask)
            
            tr_loss.append(train_error)
            te_loss.append(test_error)
            losses.append(loss)

        return current_result_new, tr_loss, te_loss

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

train = np.loadtxt('/Users/lmac/Documents/inf_search/movie-recomendation-fall-2020/train.txt',  delimiter="\t", dtype=int)

M = csr_matrix((train[:,2], (train[:,0]-1, train[:,1]-1))).toarray()



In [ ]:
np.random.seed(100)
def divide_dataset(matrix, ratio=0.1):

    pres_elements = np.arange(matrix.shape[0] * matrix.shape[1])[matrix.ravel() > 0]
    
    test_size = int(ratio * pres_elements.shape[0])
    
    np.random.shuffle(pres_elements)
    
    train_el = pres_elements[test_size:]
    test_el = pres_elements[:test_size]
    
    train_ind = np.unravel_index(train_el, matrix.shape)
    test_ind = np.unravel_index(test_el, matrix.shape)
    
    
    train_mask = np.zeros(matrix.shape)
    test_mask = np.zeros(matrix.shape)
    
    for i in range(train_ind[0].shape[0]):
        train_mask[train_ind[0][i], train_ind[1][i]] = 1
    
    for i in range(test_ind[0].shape[0]):
        test_mask[test_ind[0][i], test_ind[1][i]] = 1
        
    return matrix * train_mask, matrix * test_mask

In [ ]:
np.random.seed(100)
results = []

alphas = [25, 30, 35, 40]
embedding_sizes = [5, 6]
lamddas = [2, 5, 8]
eps = [0.05,0.1,0.15]

for alph in alphas:
    for emb_size in embedding_sizes:
        for lam in lamddas:
            for e in eps:
                print(alph, emb_size, lam, e)
                model = ImplicitALS(max_epoch=20,
                        embedding_size=emb_size,
                        alpha=alph,
                        lamdda=lam,
                        eps=e,
                        mean_decrease=0.85)
                M_train, M_v = divide_dataset(M, 0.1)
                res, _, _ = model.fit(M_train, M_v)
                results.append(res)

In [ ]:
final_result = np.zeros(results[0].shape)
for r in results:
    final_result = final_result + r
final_result = final_result/len(results)


In [ ]:
test = pd.read_csv('/Users/lmac/Documents/inf_search/movie-recomendation-fall-2020/test.txt', header = None, sep = "\t")
test.columns = ["UserId", "FilmId"]

In [ ]:

with open("submission_als_new_try.csv", 'w') as write_file:
    print("Id,Score", file=write_file)
    for test_id in range(len(test)):
            n_row = test["UserId"][test_id] - 1
            n_col = test["FilmId"][test_id] - 1
            print(f"{test_id + 1},{final_result[n_row, n_col]}", file=write_file)